<a id=0></a>
# 7.Categorical Features
カテゴリカル特徴量（変数）の取り扱い

---
### [1.LabelEncoder()](#1)
### [2.get_dummies()](#2)
### [3.OneHotEncoder()](#3)
### [4.pd.get_dummies()とOneHotEncoder()の違い](#4)
### [5.Seriesのstr属性を使う](#5)

---

データセットとしてsample1_without_index.csvを使用する

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# google colaboratoryの場合はドライブをマウントして、適切なパスを指定してください
df = pd.read_csv('./sample1_without_index.csv')
df.head()

,Date,Price,Quantity,Width,Height,Quality,Score,Difference,Color,Shape
0,1997-07-05,2291,25,2.946650,5.305868,45.893300,52.762659,0.276266,green,triangle
1,1997-07-06,506,16,1.915208,0.679004,50.611735,31.453719,-1.854628,blue,NaN
2,1997-07-07,9629,32,7.869855,6.563335,43.830416,56.239011,0.623901,blue,square
3,1997-07-08,6161,67,6.375209,5.756029,41.358007,61.453113,1.145311,green,square
4,NaN,8570,55,0.390629,3.578136,55.739709,NaN,1.037190,red,square


In [3]:
df = df[['Color', 'Shape']]

In [4]:
df.head()

,Color,Shape
0,green,triangle
1,blue,NaN
2,blue,square
3,green,square
4,red,square


In [5]:
df.isnull().sum()

Color    4
Shape    5
dtype: int64

In [6]:
df[df['Color'].isnull()].index

Index([19, 37, 40, 73], dtype='int64')

---
<a id=1></a>
[Topへ](#0)

---
## 1. LabelEncoder()  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html  
※ ラベルを数値(0, 1, 2, ...)で置換する

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
# インスタンスもしくはオブジェクトと呼ばれる
encoder = LabelEncoder()

In [9]:
# フィット、適合・学習させる
encoder.fit(df['Color'])

LabelEncoder()

In [10]:
# フィットさせた結果、Colorの持つクラスを記憶する
encoder.classes_

array(['blue', 'green', 'red', nan], dtype=object)

In [11]:
# transformで変換を行う
encoder.transform(df['Color'])

array([1, 0, 0, 1, 2, 1, 0, 2, 2, 2, 1, 0, 1, 1, 2, 0, 0, 0, 0, 3, 1, 0,
       1, 0, 1, 1, 1, 2, 1, 1, 0, 0, 0, 2, 0, 1, 1, 3, 0, 0, 3, 2, 1, 2,
       0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 1, 2, 1,
       2, 1, 1, 2, 1, 0, 1, 3, 1, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0, 2, 0,
       1, 2, 2, 2, 0, 1, 2, 0, 2, 0, 0, 2])

In [12]:
df.head()

,Color,Shape
0,green,triangle
1,blue,NaN
2,blue,square
3,green,square
4,red,square


In [13]:
# 各色とNaNを含むレコードを確認する
df_ce = df.copy()
df_ce['Color_encoded'] = encoder.transform(df['Color'])
df_ce = df_ce[['Color', 'Color_encoded', 'Shape']]
df_ce.loc[36:42]

,Color,Color_encoded,Shape
36,green,1,square
37,NaN,3,circle
38,blue,0,square
39,blue,0,square
40,NaN,3,triangle
41,red,2,NaN
42,green,1,square


In [14]:
# 変換済みのラベルを逆変換もできる
encoder.inverse_transform(df_ce.loc[36:42, 'Color_encoded'])

array(['green', nan, 'blue', 'blue', nan, 'red', 'green'], dtype=object)

---
<a id=2></a>
[Topへ](#0)

---
## 2. get_dummies()  
https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html  
※　カテゴリー変数をダミー変数化（0 or 1）する

* ダミー変数化を実行
* drop_first=Trueとは
* np.nanはどうなるのか
---

ダミー変数化を実行

In [15]:
pd.get_dummies(df['Color']).head()
# 数値ではない説明変数をそれぞれのパラメータのTrue/Failseに変換することでカテゴリカルに扱えるようにする

,blue,green,red
0,False,True,False
1,True,False,False
2,True,False,False
3,False,True,False
4,False,False,True


drop_first=Trueとは  

In [16]:
# blueのカラムがない場合でも0, "0ならばblue"と判別できる。このように説明変数説明するパラメータが重複しないようdrpo_firstを行うのが定石(正規化?)
pd.get_dummies(df['Color'], drop_first=True).head()

,green,red
0,True,False
1,False,False
2,False,False
3,True,False
4,False,True


In [18]:
df_cd = pd.get_dummies(df, columns=['Color'], drop_first=True)

In [19]:
df_cd.head()

,Shape,Color_green,Color_red
0,triangle,True,False
1,NaN,False,False
2,square,False,False
3,square,True,False
4,square,False,True


In [20]:
# 複数カラムを同時に変換、さらにデータフレーム形式で返ってくるため便利
pd.get_dummies(df, columns=['Color', 'Shape'], drop_first=True)

,Color_green,Color_red,Shape_square,Shape_triangle
0,True,False,False,True
1,False,False,False,False
2,False,False,True,False
3,True,False,True,False
4,False,True,True,False
...,...,...,...,...
95,False,False,False,False
96,False,True,False,True
97,False,False,True,False
98,False,False,False,False


np.nanはどうなるのか

In [21]:
df_cd.isnull().sum()
# nanがなくなっている

Shape          5
Color_green    0
Color_red      0
dtype: int64

In [22]:
# nanとblueの区別がつかない
df_cd.loc[36:42]

,Shape,Color_green,Color_red
36,square,True,False
37,circle,False,False
38,square,False,False
39,square,False,False
40,triangle,False,False
41,NaN,False,True
42,square,True,False


In [23]:
# dummy_naを使用することで解消
df_cd = pd.get_dummies(df, columns=['Color'], drop_first=True, dummy_na=True)
df_cd.loc[36:42]

# nanを含まないように前処理を行うのがよいが、どうしてもnanを含めざる得ない場合はこれを使用する

,Shape,Color_green,Color_red,Color_nan
36,square,True,False,False
37,circle,False,False,True
38,square,False,False,False
39,square,False,False,False
40,triangle,False,False,True
41,NaN,False,True,False
42,square,True,False,False


---
<a id=3></a>
[Topへ](#0)

---
## 3. OneHotEncoder()  
※　One-hot : ひとつが1で他は0  
※　pd.get_dummies()にはない機能を使ってダミー変数化を行う

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

デフォルトのKeyword Argument : drop=None, handle_unknown='error'

* OneHotEncoder()を使ってみる
* 複数の特徴量を変換
---

OneHotEncoder()を使ってみる

In [24]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [25]:
encoder.fit(df[['Color']])

OneHotEncoder()

In [26]:
encoder.categories_

[array(['blue', 'green', 'red', nan], dtype=object)]

In [27]:
encoder.transform(df[['Color']])
# Sparse : まばらな、疎な
# メモリの消費が減るため、sparse=Trueが初期設定となっている

<100x4 sparse matrix of type '<class 'numpy.float64'>'
	with 100 stored elements in Compressed Sparse Row format>

In [29]:
# アレイの確認方法
encoder.transform(df[['Color']]).toarray()[:5]
# drop_firstにはなっていない

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

複数の特徴量を変換

In [30]:
encoder = OneHotEncoder()

In [31]:
encoder.fit(df)

OneHotEncoder()

In [32]:
encoder.categories_

[array(['blue', 'green', 'red', nan], dtype=object),
 array(['circle', 'square', 'triangle', nan], dtype=object)]

In [33]:
encoder.transform(df).toarray()[:5]
# DataFrame形式でないのは、ndarrayをモデルにfitさせるのが効率的（無駄がない）身体と思われる。人にとってはDataFrameのほうが見やすくはあるが

array([[0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0.]])

In [34]:
encoder.inverse_transform([[0, 1, 0, 0, 0, 1, 0, 0]])

array([['green', 'square']], dtype=object)

---
<a id=4></a>
[Topへ](#0)

---
## 4. pd.get_dummies()とOneHotEncoder()の違い

* get_dummies()ではトレインセットとテストセットに差が生じる
* OneHotEncoder(handle_unknown='error', drop='first')の場合
* OneHotEncoder(handle_unknown='ignore')の場合
---

get_dummies()ではトレインセットとテストセットに差が生じる

In [35]:
np.random.seed(1)
s = pd.Series(np.random.choice([0, 1], len(df)), name='target')
s

0     1
1     1
2     0
3     0
4     1
     ..
95    0
96    1
97    1
98    0
99    1
Name: target, Length: 100, dtype: int64

In [37]:
df_new = pd.concat([df, s], axis=1)
df_new.head()

,Color,Shape,target
0,green,triangle,1
1,blue,NaN,1
2,blue,square,0
3,green,square,0
4,red,square,1


In [38]:
# トレインテストスプリット
from sklearn.model_selection import train_test_split

In [39]:
# y : 目的変数、X : 説明変数
y = df_new.pop('target')   # pop()では元のdf_newからtargetが切り出される
X = df_new  # 説明変数は複数あるので通常大文字Xと置く

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, stratify=y, random_state=17)
# 通常test_sizeは2割,3割
# stratify : 階層化。yの分離に偏りをなくす。train, testで0, 1の割合を均等にする
# random_stateはseedのようなもの

In [41]:
X_test
# testにはtrainにあるred, squareが存在しない
# この場合はどうなるのか？get_dummiesとonehot_encoderで確認

,Color,Shape
16,blue,circle
29,green,triangle
80,blue,triangle
44,blue,triangle
48,blue,triangle


In [42]:
pd.get_dummies(X_train, drop_first=True, dummy_na=True).head()

,Color_green,Color_red,Color_nan,Shape_square,Shape_triangle,Shape_nan
94,False,True,False,True,False,False
3,True,False,False,True,False,False
25,True,False,False,False,True,False
42,True,False,False,True,False,False
69,False,True,False,False,True,False


In [43]:
pd.get_dummies(X_test, drop_first=True, dummy_na=True).head()
# 一部の説明変数が削除されてしまっているため、特徴量の数、項目が変化してしまう。テストデータはモデルに当てはめるデータとして使用できない。

,Color_green,Color_nan,Shape_triangle,Shape_nan
16,False,False,False,False
29,True,False,True,False
80,False,False,True,False
44,False,False,True,False
48,False,False,True,False


In [44]:
encoder = OneHotEncoder(drop='first')

In [45]:
encoder.fit_transform(X_train).toarray()[:5]

array([[0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.]])

In [46]:
encoder.transform(X_test).toarray()
# encoderが記憶しているため、カラム数と項目は一致する

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

OneHotEncoder(handle_unknown='error', drop='first')の場合

In [47]:
# ‘error’ : Raise an error if an unknown category is present during transform.
# ‘ignore’ : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. In the inverse transform, an unknown category will be denoted as None.

In [49]:
encoder_error = OneHotEncoder(handle_unknown='error', drop='first')

In [50]:
encoder_error.fit_transform(X_train).toarray()[:5]

array([[0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.]])

In [51]:
encoder_error.transform(X_test).toarray()

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [52]:
# trainにはない値がtestに存在するという状況を作る
X_test_new = X_test.copy()
X_test_new.loc[16, 'Color'] = 'purple'
X_test_new

,Color,Shape
16,purple,circle
29,green,triangle
80,blue,triangle
44,blue,triangle
48,blue,triangle


In [56]:
encoder_error.transform(X_test_new)
# 訓練データに含まれていない値が、テストデータに含まれている場合の処理、今回は"purple"。エラーを発生させることで対応する

ValueError: Found unknown categories ['purple'] in column 0 during transform

OneHotEncoder(handle_unknown='ignore')の場合

In [63]:
encoder_ignore = OneHotEncoder(handle_unknown='ignore')
# drop='first'とは共存できない
# データ数が少ない、説明変数が多いなどの場合に訓練データにないが、テストデータに含まれるパラメータという状況が怒る可能性がある。
# そのときに使用する
# handle_unknownとdrop="first"は共存できない。drop後の数値と、unknowの値の数値が同じになってしまう可能性があるため

In [64]:
encoder_ignore.fit(X_train)

OneHotEncoder(handle_unknown='ignore')

In [65]:
encoder_ignore.transform(X_test).toarray()

array([[1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.]])

In [66]:
encoder_ignore.transform(X_test_new).toarray()
# 未知の値(今回はpurple)に対してはすべてを0とすることで区別をする

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.]])

#### 状況に応じて使い分ける（例）
* 分類される値が少ない、レコード量が多い  
    ＝＞　testデータに欠ける値はない　＝＞　get_dummies, OneHotEncoder(drop='first')
* 分類される値が少ない、レコード量が少ない  
    ＝＞　testデータに欠ける値があるかもしれない　＝＞　OneHotEncoder(handle_unknown='error', drop='first')
* 分類される値が多い、レコード量が少ない  
    ＝＞　testデータにtrainデータにない値が確実に入る　＝＞ OneHotEncoder, handle_unknown='ignore'

---
<a id=5></a>
[Topへ](#0)

---
## 5.Seriesのstr属性を使う

* Series.strとは
* メソッドを確認
* 利用頻度の高い置換、抽出、分離
---

Series.strとは

In [67]:
df = pd.DataFrame()
df['ID'] = ['A-123', 'B-456', 'A-789', 'B-123']
df['Color'] = ['py/white black', 'red green blue', 'py/yellow', 'purple white']
df

,ID,Color
0,A-123,py/white black
1,B-456,red green blue
2,A-789,py/yellow
3,B-123,purple white


In [68]:
df['ID'].str
# df.str

In [69]:
df['ID'].str[:3]

0    A-1
1    B-4
2    A-7
3    B-1
Name: ID, dtype: object

メソッドを確認

In [70]:
df['ID'].str.lower()

0    a-123
1    b-456
2    a-789
3    b-123
Name: ID, dtype: object

In [71]:
df['ID'].str.startswith('B')
# endswith

0    False
1     True
2    False
3     True
Name: ID, dtype: bool

In [72]:
df['Color'].str.contains('white')

0     True
1    False
2    False
3     True
Name: Color, dtype: bool

In [73]:
# もしくは
df['Color'].str.contains('ye|pu')

0    False
1    False
2     True
3     True
Name: Color, dtype: bool

利用頻度の高い置換、抽出、分離

In [74]:
df['Color'].str.replace('black', 'gold')

0     py/white gold
1    red green blue
2         py/yellow
3      purple white
Name: Color, dtype: object

In [75]:
df['ID'].str.split('-')

0    [A, 123]
1    [B, 456]
2    [A, 789]
3    [B, 123]
Name: ID, dtype: object

In [76]:
# expandという引数がある
df['ID'].str.split('-', expand=True)
# splitしつつ別のカラムに設定できる

,0,1
0,A,123
1,B,456
2,A,789
3,B,123


In [77]:
df[['ID_a', 'ID_n']] = df['ID'].str.split('-', expand=True)
df

,ID,Color,ID_a,ID_n
0,A-123,py/white black,A,123
1,B-456,red green blue,B,456
2,A-789,py/yellow,A,789
3,B-123,purple white,B,123


In [78]:
df[['Color_1', 'Color_2', 'Color_3']] = df['Color'].str.split(' ', expand=True)
df

,ID,Color,ID_a,ID_n,Color_1,Color_2,Color_3
0,A-123,py/white black,A,123,py/white,black,None
1,B-456,red green blue,B,456,red,green,blue
2,A-789,py/yellow,A,789,py/yellow,None,None
3,B-123,purple white,B,123,purple,white,None


In [79]:
# extractでは正規表現を使うことも可能
df['Color_1'].str.extract('(py/)', expand=True)

,0
0,py/
1,NaN
2,py/
3,NaN


In [80]:
df['py'] = df['Color_1'].str.extract('(py/)', expand=True)
df

,ID,Color,ID_a,ID_n,Color_1,Color_2,Color_3,py
0,A-123,py/white black,A,123,py/white,black,None,py/
1,B-456,red green blue,B,456,red,green,blue,NaN
2,A-789,py/yellow,A,789,py/yellow,None,None,py/
3,B-123,purple white,B,123,purple,white,None,NaN


In [81]:
df['Color_1'] = df['Color_1'].str.replace('py/', '')
df

,ID,Color,ID_a,ID_n,Color_1,Color_2,Color_3,py
0,A-123,py/white black,A,123,white,black,None,py/
1,B-456,red green blue,B,456,red,green,blue,NaN
2,A-789,py/yellow,A,789,yellow,None,None,py/
3,B-123,purple white,B,123,purple,white,None,NaN


---
[Topへ](#0)

---
## 以上
    
---